# Using foreign keys in SQL

In the previous notebook, *09.1 Defining Foreign Keys in SQL*, we showed how foreign keys can be defined on tables to represent the relationships that hold between entities.  In this notebook, we will look a bit more closely at the behaviour of foreign keys, and see how they are used to maintain referential integrity in a database.

## Setting up

The next group of cells set up your database connection, and reset the database to a clean state. Check notebook *08.1 Data Definition Language in SQL* if you are unsure what the next cells do.

You may need to change the given values of the variables `DB_USER` and `DB_PWD`, depending on which environment you are using.

In [ ]:
# Make the connection

%run sql_init.ipynb
print("Connecting with connection string : {}".format(DB_CONNECTION))
%sql $DB_CONNECTION

In [ ]:
%run reset_databases.ipynb

## The `hospital` schema

In notebooks 08.1, 08.2, 08.3 and 09.1, we started with an empty database, which we used so that we could demonstrate the creation and deletion of tables, and the manipulation of data in those tables.

When databases become larger and more complex, it is often necessary to organise the database tables into logical groups to make them more manageable. A database is organised into a collection of *schemas*, where each schema contains a number of tables, as well as the constraints defined on those tables.

In this notebook and the next, we have defined a schema, `hospital`. This schema contains the four populated tables `patient`, `doctor`, `drug` and `prescription`, as well as the primary key constraints and foreign key constraints defined on them, as in Activity 3 of notebook *09.1 Defining Foreign Keys in SQL*.

To access the tables in a particular schema, we can use the qualified name of the table, where we make the schema explicit. For example, to see the `patient` table in the `hospital` schema, we can call the table as `hospital.patient`:

In [ ]:
%%sql

SELECT *
FROM hospital.patient;

As a comparison, if we try a `SELECT` query on the unqualified table `patient`, we should find that an error is raised, stating that the table does not exist (assuming that the reset script was run in the preamble for this notebook):

In [ ]:
%%sql

SELECT *
FROM patient;

In fact, the default schema is called `public`: this is the schema used if no other is specified.

Of course, it becomes rather tedious to have to qualify every table name with the schema that contains it, and so we can tell PostgreSQL which schemas to use first. PostgreSQL has a built-in variable `search_path` which gives the order of schemas to search for tables whose names are not qualified with a schema. To tell PostgreSQL to search the `hospital` schema before the `public` schema, we use the following:

In [ ]:
%%sql

SET search_path TO hospital, public;

We can see the value of `search_path` with:

In [ ]:
%%sql

SHOW search_path;

which should list `hospital` before `public`.

If we now attempt to `SELECT` from the `patient` table, we should find that the `hospital.patient` table is queried, rather than the `public.patient` table:

In [ ]:
%%sql

SELECT *
FROM patient

Should you need it, you can check the [entry on schemas in thePostgreSQL manual](https://www.postgresql.org/docs/9.5/ddl-schemas.html).

## Working With Constraints


There was a lot of material in notebook *09.1 Defining Foreign Keys in SQL*, but this notebook will revise some of the key ideas from the perspective of working with relational databases.

### Constraints can prevent table alterations if they would be violated


As we saw in [Part 9, section 4](https://learn2.open.ac.uk/mod/oucontent/olinkremote.php?website=TM351&targetdoc=Part%209%20Relational%20data%20modelling&targetptr=4.1), referential integrity refers to the constraint that "every non `NULL` foreign key value must match an existing primary key value". It is important because these constraints ensure that any cross-references made from one table to another are consistent. Referential integrity allows this consistency to be maintained if changes are made to the database.


Let's check the `patient` table to see what the consistent cross-references to `patient_id` values could be; the following query returns all the unique values of `patient_id` in the `patient` table:

In [ ]:
%%sql 

SELECT DISTINCT patient_id 
FROM patient;

### Activity 1

Why is the use of `DISTINCT` in the above query effectively redundant?

Write your answer in this cell

#### Our solution

To reveal our solution, run this cell or click on the triangle symbol on the left-hand side of the cell.

`patient_id` is the (whole) primary key of the `patient` table. We have seen throughout Part 8 that the primary key of a table must not be `NULL` and may not contain repeated values. Therefore, all the values in `patient_id` are constrained to be `DISTINCT`.

#### End of Activity 1

----------------------------------------------------

Now let's see how these values affect the possible data in rows in the `prescription` table. First, use a `SELECT` query to look at the rows in `prescription`:

In [ ]:
%%sql

SELECT *
FROM prescription;

Let's add a new row to the `prescription` table that references one of those `patient_id` values. The row we will add is:

|patient_id|doctor_id|drug_code|date    |dosage  |duration|
|----------|---------|---------|--------|--------|--------|
|p001|d06|T02378|2014-08-11|50 mg 3 x day|As required|


which you can see from the previous `SELECT` statement does not already exist in the `prescription` table.

An `INSERT` statement will add the row:

In [ ]:
%%sql
INSERT INTO prescription (patient_id, date, doctor_id, drug_code, dosage, duration)
    VALUES ('p001', '2014-08-11', 'd06', 'T02378', '50 mg 3 x day', 'As required');

In [ ]:
%%sql

SELECT * 
FROM prescription;

The `SELECT` query shows that the row has been added as required.

Now what happens if we try to add a row to the prescription table that contains a `patient_id` that is not in the `patient` table? Let's try an `INSERT INTO prescription` statement that uses a value of `patient_id` that does not appear in the `patient` table:

In [ ]:
%%sql

INSERT INTO prescription (patient_id, date, doctor_id, drug_code, dosage, duration)
VALUES ('Sam', '2017-07-01', 'd06', 'A12458', '10 mg 5 x day', '10 days');

 The SQL statement should have raised an `IntegrityError`. Looking at the error message closely, we see the explanation:
 
 `DETAIL:  Key (patient_id)=(Sam ) is not present in table "patient".`
 
The constraint checker has identified that `Sam` is not a recognised `patient_id` in the `patient` table. In enforcing the constraint, it has prevented the addition of a row into the `prescription` table that tries to reference that value in the `patient` table.
 

### Foreign key columns with missing values

If the foreign key value is `NULL`, then by default a constraint violation does *not* occur.

For example, suppose that a new patient were to be admitted to the hospital, but had not been assigned a doctor. We might want to be able to add the patient's data without (yet) adding a reference to a doctor. Remember that the `patient` table contains a foreign key reference to the `doctor` table via a foreign key.

We can add the new patient's information with an `INSERT` statement, which does not include a value for `doctor_id`:

In [ ]:
%%sql

INSERT INTO patient(patient_id, patient_name, date_of_birth, gender, height_cm, weight_kg)
    VALUES ('p028', 'Huynh', '1981-07-03', 'F', 155, 45.6);

You should have found that this has not raised an error. We can check that the row has been added to the `patient` table:

In [ ]:
%%sql

SELECT *
FROM patient
WHERE patient_id='p028';

<div class='alert alert-info'>`NULL` items in the database are represented as `None` or `NaN` in a pandas dataframe.</div>

The row has been added, with no value in the `doctor_id` column. 

Of course, if we tried to update the `doctor_id` for the new patient, we would have to ensure that there was a corresponding value in the `doctor` table. For example, if we try to update the value so that it refers to a (non-existent) doctor with id of `doc`, then an error will be raised:

In [ ]:
%%sql

UPDATE patient
    SET doctor_id='doc'
    WHERE patient_id='p028';

If you try to execute the previous cell, you should receive an `IntegrityError`, stating that the `insert or update on table "patient" violates foreign key constraint "hospital_patient_doctor_fk"`. If we check the row for this patient, then we will see that the table has not been updated: the row still has a missing value for `doctor_id`:

In [ ]:
%%sql

SELECT *
FROM patient
WHERE patient_id='p028';

If we try the same code with a value for `doctor_id` which does appear in the `doctor_id` column of the `doctor` table, then the update should be carried out without an error. Let's try to update the row for the new patient so that it contains a reference to the doctor with id `d09`, which *does* exist in the database:

In [ ]:
%%sql

UPDATE patient
    SET doctor_id='d09'
    WHERE patient_id='p028';

The update is successful, and we can now see that the row has a value for `doctor_id`:

In [ ]:
%%sql

SELECT *
FROM patient
WHERE patient_id='p028';

### Mandatory and optional participation in relationships

In order to prevent the addition of `NULL` values in the foreign key column, we need to add a `NOT NULL` constraint definition to the table for that column. (The different types of relationship, including mandatory and optional participation, are discussed in [Part 9, section 3](https://learn2.open.ac.uk/mod/oucontent/olinkremote.php?website=TM351&targetdoc=Part%209%20Relational%20data%20modelling&targetptr=3).)

If the foreign key column can contain missing values (`NULL`s), then the referencing table's participation in the relationship is *optional*. In the case of `patient` and `doctor`, the ERD would show this as:

![Patient to doctor foreign key with optional participation](patient-doctor-fk_optional.jpg)

That is, each patient is associated with either zero or one doctors. Patients who are not associated with a doctor would not have a value in the `patient.doctor_id` column.

Alternatively, if we set the `patient.doctor_id` column to be `NOT NULL`, then the participation of `patient` in the relationship is *mandatory*:

![Patient to doctor foreign key with mandatory participation](patient-doctor-fk_mandatory.jpg)

That is, each patient is associated with exactly one doctor.

To see this, we can add a `NOT NULL` constraint to the foreign key column (we saw how to add a `NOT NULL` constraint in notebook *08.3 Adding column constraints to tables*):

In [ ]:
%%sql

ALTER TABLE patient
ADD CONSTRAINT hospital_patient_doctor_fk_not_null
     CHECK (doctor_id IS NOT NULL);

Note that, as we would expect, executing the previous cell will raise an error if there are currently missing values in the `doctor_id` column.

If the cell ran without raising an error, we should now find that it is not possible to add a new row to the `patient` table which does not contain a value for `doctor_id`:

In [ ]:
%%sql

INSERT INTO patient(patient_id, patient_name, date_of_birth, gender, height_cm, weight_kg)
    VALUES ('p029', 'Garcia', '1980-03-13', 'M', 187, 82.6);

Executing the cell should result in an `IntegrityError` being raised, with the explanation `new row for relation "patient" violates check constraint "patient_doctor_fk_not_null"`.

The participation of `patient` in the relationship is now mandatory.

### Activity 2

In Activity 3 of notebook *09.1 Defining Foreign Keys in SQL*, we created the `prescription` table from a dataframe `prescription_df` and defined an additional set of constraints on the new table with the SQL code:

```python
prescription_df.to_sql('prescription', DB_HOSPITAL_CONNECTION, if_exists='replace', index=False)
```

```sql
ALTER TABLE prescription
ADD CONSTRAINT prescription_pk
    PRIMARY KEY (patient_id, doctor_id, drug_code, date);```
    
```sql
ALTER TABLE prescription
ADD CONSTRAINT prescription_patient_fk
    FOREIGN KEY (patient_id) REFERENCES patient;```
    
```sql
ALTER TABLE prescription
ADD CONSTRAINT prescription_doctor_fk
    FOREIGN KEY (doctor_id) REFERENCES doctor;```
    
```sql
ALTER TABLE prescription
ADD CONSTRAINT prescription_drug_fk
    FOREIGN KEY (drug_code) REFERENCES drug;```

Why can none of the foreign keys defined on the `prescription` table be `NULL`?

Write your answer in this cell

#### Our solution

To reveal our solution, run this cell or click on the triangle symbol on the left-hand side of the cell.

Each of the `prescription` table's foreign key columns (`patient_id`, `doctor_id` and `drug_code`) also forms part of the primary key (`(patient_id, doctor_id, drug_code, date)`). Although by default a foreign key can contain `NULL` values, the primary key may not, and so in the `prescription` table, any `NULL` values in a foreign key column would constitute a violation of the primary key constraint.

#### End of Activity 2

-------------------------------------------------------------

## Using Constraints To Support Data Cleaning

One of the problems associated with cleaning a dataset is identifying the problems with it, particularly if it contains thousands or even millions of rows spread across multiple tables. By defining appropriately defined constraints on database tables and then trying to load data into them, constraint violations can help identify at least some of the dirty data records.

### Constraints Can Help Catch Dirty Data

A useful consequence of constraints is that they can help catch various forms of dirty data.

For example, suppose we have a file containing some new data to be entered into the `prescription` table. We have put this data in the `dirty_prescription.csv` file in this folder. Let's look at the contents of the file:

In [ ]:
!cat dirty_prescription.csv

As we have done before, we can read this data into a dataframe:

In [ ]:
new_prescription_df=pd.read_csv('dirty_prescription.csv', 
                                parse_dates=['date'])
new_prescription_df

Now we should be able to use the dataframe's `.to_sql` method to add these new rows to the database. Note a couple of important aspects of this call to the `.to_sql` method:
1. we use `if_exists='append'` as a parameter which adds the data to the table, rather than completely replacing the table, and
2. when using `.to_sql` we need to explicitly state the schema used (the default is `public`), so in this case we need to include a parameter `schema='hospital'`.

In [ ]:
new_prescription_df.to_sql('prescription',
                           DB_CONNECTION,
                           schema='hospital',
                           if_exists='append',
                           index=False
                           )

If you ran the previous cell, you should have found that an `IntegrityError` was raised, with the message that `Key (drug_code)=(C3l319) is not present in table "drug".`.

So what went wrong? You might (if you look hard) be able to see the problem: in this case it is that the value provided for the foreign key is `C3l319`, whereas the correct value should be `C31319`. That is, the third character of the code should be the number `1`, rather than the letter `l`. (The standard notebook font is not exactly helpful in distinguishing between these characters! If you zoom the screen in, you may be able to see the difference more clearly.)

In [ ]:
# If you can't see the difference, test the values
'C31319' == 'C3l319'

This kind of transcription error is common, and can be caught if constraints are correctly set up.

Let's change the `new_prescription_df` dataframe so that this drug code is correct:

In [ ]:
new_prescription_df=pd.read_csv('dirty_prescription.csv', 
                                parse_dates=['date'])

#The .at[INDEX,COLUMN] dataframe method allows you to set the value in a specified cell
new_prescription_df.at[0,'drug_code'] = 'C31319'
new_prescription_df

Now that we've updated the offending value of `drug_code`, we should be able to add the data from the dataframe (again remembering to make the `hospital` schema explicit):

In [ ]:
new_prescription_df.to_sql('prescription',
                           DB_CONNECTION,
                           schema='hospital',
                           if_exists='append',
                           index=False
                           )

Again, we have an error raised. What has gone wrong this time? Looking at the error message closely, we see the error:

`Key (doctor_id)=( d07) is not present in table "doctor".`

The space in the key value suggests that we are trying to enter a key value of <code> d07</code> rather than the `d07` which exists in the column `doctor_id`, the primary key of the `doctor` table. The presence of the leading whitespace makes all the difference. In this case, we have dirty data in the CSV file that is propagated into the dataframe. The constraint that the `doctor_id` values in the `prescription` table match corresponding values in the referenced `doctor` table prevents the prescription item being added because there is no matching record in the `doctor` table.

Again, this is the sort of dirty data that is easily missed. Data can often look as though it is in the correct form, but subtle differences between representations can cause difficulties when we try to combine the data. This is especially true when we are dealing with datasets containing thousands, or even millions of instances: it is impractical to try to identify discrepancies manually, and so functionality such as integrity maintenance can avoid problems later in the analysis.

<div class='alert alert-danger'>Renderers such as the Jupyter notebook markdown parser and renderer may also display something other than what was actually written. For example, double click on this cell to reveal the different "source" representation of the following three values:
<br/><br/>

`    d01`, `d01` and `           d01           `
<br/><br/>
To preserve the whitespace in a markdown code string, use a &lt;code&gt;&lt;/code&gt; block rather than backticks: <code>   d01</code></div>

# Foreign keys and `JOIN`s


If there are any foreign key references defined across database tables, we can be safe in the knowledge that each row in the child table (the table containing the foreign key) __will__ match exactly one referenced item in the parent table via the foreign key.

You should already be familiar with the mechanics of joining multiple tables using SQL from working through notebook *03.3 Combining data from multiple datasets*. Let's recap on the syntax used to join tables. Remember, this has nothing to do with the presence, or otherwise, of foreign key relationships, although such relationships may inform us how information about particular entities is distributed about the database:

<code>SELECT * 
FROM child_table JOIN parent_table
    USING (common_column_name);</code>

If the column names don't match (or even if they do), we can use the more general formulation:

<code>SELECT * 
FROM child_table JOIN parent_table
    ON child_table.fk_column_name = parent_table.pk_column_name;
</code>



In notebook *03.3 Combining data from multiple datasets*, you learned how to join tables that were unconstrained in terms of foreign key relationships. The presence or otherwise of foreign keys relationships is irrelevant to the operation of joining tables: *foreign keys are used to constrain data in order to maintain the integrity of the database*.

However, in many cases, we *are* likely to join tables on columns that are linked by foreign key relationships.

### Activity 3


If we can join tables anyway, without the need for defining primary or foreign keys, what benefits do joining tables using columns that form foreign key references bring to queries over "arbitrarily" joined tables?

Write your answer in this cell

#### Our solution

To reveal our solution, run this cell or click on the triangle symbol on the left-hand side of the cell.

The foreign key references clearly identify that tables are related and which columns relate them. In a less structured database, the person writing the query must make assumptions about how the tables are related, and the columns that relate them. For example, this might be based on observing similarities between the contents, and perhaps the names, of the columns.

For content added to the database without constraints, data that might be expected to be discoverable from a child table joined to a parent table may be missing from the parent table. If a foreign key constraint had been in place when the data was added to the child table, that situation would have caused a constraint violation.

#### End of activity 3

-------------------------------------------------------

## Joining Tables Without a Foreign Key Reference



As has been mentioned, the *ability* to join tables has nothing to do with existence, or otherwise, of foreign key references between joined tables.

For example, if you are working with data loaded into a database from a set of *pandas* dataframes, a process that dynamically creates database tables without the existence of either primary or foreign keys, there won't be any foreign key relationships in the database. However, there might be things spread across the data tables that we could recognise as attributes of a common entity and which we *could* associate using foreign keys.

In many databases, there are also likely to be columns that can provide the basis for joins but that we are not likely to want to associate using foreign keys. Dates and locations are a good example of this, providing a basis for correlating distinctly represented items on the basis that they occurred on the same date, in the same location, or at the same date and time.

Identifying columns that might be sensibly joined together is one of the key skills associated with performing many data investigations. Joinable columns across multiple datasets also provides a vector for attack. For example, it may lead to sensitive information being revealed through mosaic intelligence style attacks, in which information from multiple independent sources is pieced together, or allow for anonymised records in one dataset to be deanonymised by reference to another.

Looking at the tables in the `hospital` schema, there are several columns that we might attempt to join in order to learn more about the entities in the database than the database designer might have intended. For example, suppose that we know one of the doctors is also a patient, but that they use their professional name at work, and another name in their personal life, such as when they're a patient.

A search on common names across the `patient` and `doctor` tables might turn up nothing that allows us to match a doctor with a patient record:

In [ ]:
%%sql

SELECT * 
FROM patient JOIN doctor
    ON patient.patient_name=doctor.doctor_name;

But if the doctor table includes a birth date column, a search that tries to match *birth dates* might give us an investigative lead (or it might just be a coincidence):

<code>SELECT * 
FROM patient JOIN doctor
    ON patient.date_of_birth=doctor.date_of_birth;
</code>

## Different Types of JOIN



As you should recall from notebook *03.3 Combining data from multiple datasets*, the `JOIN` operator takes several different forms. By default, the PostgreSQL `JOIN` operator uses an `INNER JOIN`, but other join types are possible.

- `INNER JOIN`
- `LEFT OUTER JOIN`
- `RIGHT OUTER JOIN`
- `FULL OUTER`


### Activity 4

For what sorts of query, or under what circumstances, might you use each type of join? To what extent does the order tables are specified in the join clause matter?

Write your answer in this cell

#### Our solution

To reveal our solution, run this cell or click on the triangle symbol on the left-hand side of the cell.

The *inner join* is one of the most commonly used joins and the one we might naturally think of as a "join" or merge. The order of the tables in the join statement is irrelevant to the join. The inner join is used to create rows that effectively straddle both tables, pulling columns from each table on the columns used to join the tables.

The *outer* joins allow the creation of result sets where we don't mind if there is no reference to link the tables being joined.  The table order does matter in cases of left or right outer joins, because the handedness of the join determines whether or not rows from the left hand table (the one to the left of the `JOIN` statement or the right hand table are included if there is no match. (Null values will be used to fill the columns "returned" from the unmatched rows in the specified table, left or right.)

Outer joins are used when we want to keep a full complement of rows from one table (or both, in the case of the full outer join). For example, if we are annotating the rows in one table with "optional" information that may have been recorded in another, but which may not be available for a particular item, an outer join would be appropriate.

#### End of Activity 4

---------------------------------------------

## What next?

You have now seen how to use foreign keys to implement relationships between entities, and how foreign keys can be used to maintain integrity in the database. In the final notebook of this part, notebook *09.3 Working With FOREIGN KEY Constraints*, you will look at some more functionality that SQL provides to maintain integrity when foreign keys are being used.